## G8 - Lavoro e Disoccupazione

In [1]:
# Import librerie per analisi dati (Pandas) e dati Istat
import os
import pandas as pd
import numpy as np
from IPython.core.display import HTML
import istat
import jsonstat

# cache dir per velocizzare analisi in locale
cache_dir = os.path.abspath(os.path.join("..", "tmp/od_la_grande_fuga", "istat_cached"))
istat.cache_dir(cache_dir)
istat.lang(0)  # lingua italiano

dir_df = os.path.join(os.path.abspath(''),'stg')

In [6]:
area_lavoro = istat.area('LAB')
area_lavoro.datasets()

cod,name,dim
DCCV_COMPL,Indicatori complementari,12
DCCV_DISOCCUPT,Disoccupati,10
DCCV_DISOCCUPTDE,Disoccupati - dati destagionalizzati,7
DCCV_DISOCCUPTMENS,Disoccupati - dati mensili,8
DCCV_FORZLV,Forze di lavoro,8
DCCV_FORZLVDE,Forze di lavoro - dati destagionalizzati,7
DCCV_FORZLVMENS,Forze lavoro - dati mensili,8
DCCV_INATTIV,Inattivi,11
DCCV_INATTIVDE,Inattivi - dati destagionalizzati,7
DCCV_INATTIVMENS,Inattivi - dati mensili,8


In [60]:
# Indicatori di lavoro - Tasso Disoccupazione
ds_lavoro = area_lavoro.dataset('DCCV_TAXDISOCCU')
ds_lavoro.dimension('Classe di età')

'Classe di età' (3:'20-24 anni', 4:'15-24 anni', 8:'25-34 anni', 9:'35-54 anni', 11:'35 anni e più', 14:'35-44 anni', 17:'45-54 anni', 19:'55 anni e più', 21:'55-64 anni', 22:'15-64 anni', 28:'15 anni e più', 30:'15-29 anni', 31:'15-74 anni', 32:'18-29 anni')

In [62]:
# NORD
spec_n = {
    "Territorio":3,
    "Sesso":3,
    "Cittadinanza":3,
    "Durata disoccupazione":3,
    "Titolo di studio":12,
    "Classe di età":28
}

#SUD
spec_s = {
    "Territorio":88,
    "Sesso":3,
    "Cittadinanza":3,
    "Durata disoccupazione":3,
    "Titolo di studio":12,
    "Classe di età":28
}

In [75]:
def disoccupati(ds, spec,nome):
    c = ds.getvalues(spec)
    ds = c.dataset(0)
    df = ds.to_data_frame('Tempo e frequenza')
    df.reset_index(level=0, inplace=True)
    df.columns = ['Anno',nome]
    df = df[(df['Anno'].str.contains("T")==False)]
    return df

In [80]:
df_s = disoccupati(ds_lavoro, spec_s, 'Sud')
df_n = disoccupati(ds_lavoro, spec_n, 'Nord')

In [83]:
df = pd.merge(df_n,df_s, how='inner', on=['Anno', 'Anno'])
df = df.set_index(['Anno'])
df.head(5)

,Nord,Sud
Anno,,
1977,5.76,8.05
1978,6.09,7.98
1979,6.14,8.60
1980,6.15,8.89
1981,6.76,9.44


In [84]:
df_filename = r'df_g8.pkl'
df_fullpath = os.path.join(dir_df, df_filename)
df.to_pickle(df_fullpath)